In [28]:
import pandas as pd
import spacy
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns


In [29]:
# Load the SOTU data
sou = pd.read_csv("data/SOTU.csv")
print(f"Total speeches: {len(sou)}")

# Subset for speeches from 2000 onwards (as required by Part 2)
sou_2000 = sou[sou['Year'] >= 2000].copy()
print(f"Speeches from 2000 onwards: {len(sou_2000)}")
sou_2000.head()

Total speeches: 246
Speeches from 2000 onwards: 25


,President,Year,Text,Word Count
0,Joseph R. Biden,2024.0,"\n[Before speaking, the President presented hi...",8003
1,Joseph R. Biden,2023.0,\nThe President. Mr. Speaker——\n[At this point...,8978
2,Joseph R. Biden,2022.0,"\nThe President. Thank you all very, very much...",7539
3,Joseph R. Biden,2021.0,\nThe President. Thank you. Thank you. Thank y...,7734
4,Donald J. Trump,2020.0,\nThe President. Thank you very much. Thank yo...,6169


In [30]:
def preprocess_text(text): 
    doc = nlp(text) 
    return [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space and len(token.lemma_) > 3]

In [31]:
# Process all texts - note this takes ~ 5 minutes to run
processed_docs = sou['Text'].apply(preprocess_text)

In [32]:
print(processed_docs[:5])



0    [speak, president, present, prepared, remark, ...
1    [president, speaker, point, president, turn, f...
2    [president, thank, thank, thank, madam, speake...
3    [president, thank, thank, thank, good, mitch, ...
4    [president, thank, thank, thank, madam, speake...
Name: Text, dtype: object


In [33]:
from gensim.corpora import Dictionary

dictionary = Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5)  
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

ImportError: cannot import name 'triu' from 'scipy.linalg' (/srv/conda/envs/notebook/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [34]:
from gensim.models import LdaModel
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=5,       
                     random_state=42,    
                     passes=10,         
                     alpha='auto',       
                     eta='auto') 


ImportError: cannot import name 'triu' from 'scipy.linalg' (/srv/conda/envs/notebook/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [35]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}\n")

NameError: name 'lda_model' is not defined